# Predicción de Ventas de Productos con Modelos de Machine Learning

Este proyecto busca predecir la cantidad de ventas aproximadas de productos de Amazon, utilizando características como precio, descuento, calificación y categoría.

Los modelos utilizados son **XGBoost** y **LightGBM**, optimizados con **RandomizedSearchCV** para encontrar la mejor combinación de hiperparámetros.

In [ ]:

import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Cargar el dataset desde Kaggle
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "karkavelrajaj/amazon-sales-dataset", "amazon.csv")

# Mostrar información básica
print("Dataset cargado correctamente")
print("Dimensiones:", df.shape)
print("Tipos de datos:")
print(df.dtypes)

# Vista previa
df.head()


In [ ]:

# Limpiar columnas numéricas
df['discounted_price_num'] = df['discounted_price'].replace({'[₹,]': ''}, regex=True).astype(float)
df['actual_price_num'] = df['actual_price'].replace({'[₹,]': ''}, regex=True).astype(float)
df['discount_percentage_num'] = df['discount_percentage'].str.replace('%', '', regex=False).astype(float)
df['rating_num'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating_count_num'] = df['rating_count'].replace({',': ''}, regex=True).astype(float)

# Eliminar filas con NaN en las columnas clave
df.dropna(subset=['rating_num', 'rating_count_num'], inplace=True)

# Mostrar las primeras filas tras la limpieza
df[['discounted_price_num', 'actual_price_num', 'discount_percentage_num', 'rating_num', 'rating_count_num']].head()


In [ ]:

# Extraer la categoría principal
df['main_category'] = df['category'].str.split('|').str[0]

# Ver las categorías principales únicas
print("Categorías principales únicas:", df['main_category'].unique())


In [ ]:

import matplotlib.pyplot as plt

# Histograma de ventas (rating_count_num)
plt.figure(figsize=(8, 6))
plt.hist(df['rating_count_num'], bins=50, color='skyblue', edgecolor='black')
plt.yscale('log')
plt.title('Distribución de las ventas (Rating Count)', fontsize=14)
plt.xlabel('Número de valoraciones (ventas)', fontsize=12)
plt.ylabel('Frecuencia (escala logarítmica)', fontsize=12)
plt.show()

# Correlación entre las principales variables
corr = df[['discounted_price_num', 'actual_price_num', 'discount_percentage_num', 'rating_num', 'rating_count_num']].corr()

# Mostrar la matriz de correlación
plt.figure(figsize=(8, 6))
plt.imshow(corr, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title('Matriz de Correlación de Variables', fontsize=14)
plt.show()


In [ ]:

# Selección de las características a usar
X = df[['discounted_price_num', 'discount_percentage_num', 'rating_num', 'main_category']].copy()

# One-hot encoding para la categoría
X = pd.get_dummies(X, columns=['main_category'], drop_first=True)

# Variable objetivo (rating_count_num)
y = df['rating_count_num']

# Ver las primeras filas para verificar la codificación
print(X.head())


In [ ]:

from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Definir el modelo XGBoost
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Realizar validación cruzada con 5 pliegues
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

# Mostrar el RMSE promedio
mean_rmse = -cv_scores.mean()
print("RMSE promedio de XGBoost con validación cruzada de 5 pliegues:", mean_rmse)


In [ ]:

from sklearn.model_selection import RandomizedSearchCV

# Espacio de búsqueda para RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'gamma': [0, 1],
    'alpha': [0, 0.1],
    'lambda': [0, 0.1]
}

# Crear el modelo XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Realizar búsqueda aleatoria con 100 combinaciones
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=100, cv=3, scoring='neg_root_mean_squared_error', random_state=42, verbose=1, n_jobs=-1)

# Ajustar el modelo
random_search.fit(X, y)

# Mostrar los mejores parámetros encontrados
print("Mejores hiperparámetros encontrados:", random_search.best_params_)


In [ ]:

# Reentrenar el modelo XGBoost con los mejores parámetros
best_xgb = random_search.best_estimator_
best_xgb.fit(X, y)

# Realizar predicciones
y_pred = best_xgb.predict(X)

# Calcular las métricas de evaluación
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)


In [ ]:

import lightgbm as lgb

# Crear el modelo LightGBM
lgb_model = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=100)

# Entrenamiento del modelo LightGBM
lgb_model.fit(X, y)

# Realizar predicciones con el modelo LightGBM
y_pred_lgb = lgb_model.predict(X_test)

# Calcular las métricas de evaluación para LightGBM
mae_lgb = mean_absolute_error(y_test, y_pred_lgb)
rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
r2_lgb = r2_score(y_test, y_pred_lgb)

# Mostrar las métricas para LightGBM
print("LightGBM -> MAE:", mae_lgb, "RMSE:", rmse_lgb, "R²:", r2_lgb)

# Comparación con el modelo XGBoost
# Entrenar el modelo XGBoost (como ya lo tenemos entrenado)
y_pred_xgb = best_xgb.predict(X_test)

# Calcular las métricas de evaluación para XGBoost
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

# Mostrar las métricas para XGBoost
print("XGBoost -> MAE:", mae_xgb, "RMSE:", rmse_xgb, "R²:", r2_xgb)

# Comparación de resultados
print("
Comparación de Modelos:")
print(f"LightGBM vs XGBoost: 
MAE: {mae_lgb} vs {mae_xgb} 
RMSE: {rmse_lgb} vs {rmse_xgb} 
R²: {r2_lgb} vs {r2_xgb}")



## Conclusiones

- **XGBoost optimizado** con **RandomizedSearchCV** mostró el mejor rendimiento en términos de **RMSE** y **R²**.
- **LightGBM** también tiene un buen rendimiento, pero XGBoost superó a LightGBM en **RMSE** y **R²**.
- Las características más importantes para predecir las ventas fueron **precio con descuento** y **calificación**.
- Las recomendaciones para mejorar el rendimiento incluyen ajustar los **porcentajes de descuento** y mejorar las **calificaciones** de los productos.

## Recomendaciones para el Negocio

1. **Mejorar la calificación de los productos**.
2. **Aplicar descuentos estratégicos** en productos que necesitan un empujón en ventas.
3. **Optimizar las categorías de productos** para identificar las más rentables.
